In [17]:
import re

# Helper function to extract all integers from a string
def extract_integers(s):
    return list(map(int, re.findall(r'\d+', s)))

# Load and parse the input file
with open('/content/AoC_2023_Day19.txt') as file:
    ll = file.read().strip().split('\n\n')
    workflow_data, parts_data = ll

# Process parts data
parts = [extract_integers(line) for line in parts_data.split("\n")]

# Process workflow data
workflow = {line.split("{")[0]: line.split("{")[1][:-1] for line in workflow_data.split("\n")}

# Recursive function to evaluate whether a part is accepted or rejected
def evaluate_part(part, workflow_name):
    workflow_rules = workflow[workflow_name]
    x, m, a, s = part
    for rule in workflow_rules.split(","):
        if rule == "R":
            return False  # Part is rejected
        if rule == "A":
            return True  # Part is accepted
        if ":" not in rule:
            return evaluate_part(part, rule)  # Recursively evaluate in another workflow
        condition, next_workflow = rule.split(":")
        if eval(condition):  # Evaluate the condition
            if next_workflow == "R":
                return False  # Part is rejected
            if next_workflow == "A":
                return True  # Part is accepted
            return evaluate_part(part, next_workflow)  # Recursively evaluate in the next workflow
    raise Exception(f"Unexpected workflow: {workflow_rules}")

# Part 1: Sum the ratings of all accepted parts
def part1():
    total_sum = 0
    for part in parts:
        if evaluate_part(part, 'in'):
            total_sum += sum(part)
    print(f"Total sum of ratings for accepted parts: {total_sum}")

# Function to refine ranges based on a condition
def refine_ranges(character, greater_than, value, ranges):
    char_index = 'xmas'.index(character)
    refined_ranges = []
    for rng in ranges:
        rng = list(rng)
        lo, hi = rng[char_index]
        if greater_than:
            lo = max(lo, value + 1)
        else:
            hi = min(hi, value - 1)
        if lo > hi:
            continue
        rng[char_index] = (lo, hi)
        refined_ranges.append(tuple(rng))
    return refined_ranges

# Recursive function to calculate accepted ranges
def calculate_accepted_ranges(workflow_name):
    return evaluate_workflow_rules(workflow[workflow_name].split(","))

# Function to evaluate workflow rules and return accepted ranges
def evaluate_workflow_rules(rules):
    current_rule = rules[0]
    if current_rule == "R":
        return []
    if current_rule == "A":
        return [((1, 4000), (1, 4000), (1, 4000), (1, 4000))]  # Full range for all ratings
    if ":" not in current_rule:
        return calculate_accepted_ranges(current_rule)

    condition, next_workflow = current_rule.split(":")
    greater_than = ">" in condition
    character = condition[0]
    value = int(condition[2:])
    value_inverted = value + 1 if greater_than else value - 1

    true_ranges = refine_ranges(character, greater_than, value, evaluate_workflow_rules([next_workflow]))
    false_ranges = refine_ranges(character, not greater_than, value_inverted, evaluate_workflow_rules(rules[1:]))

    return true_ranges + false_ranges

# Part 2: Calculate the number of distinct combinations of ratings that are accepted
def part2():
    total_accepted_combinations = 0
    for accepted_range in calculate_accepted_ranges('in'):
        combinations = 1
        for lo, hi in accepted_range:
            combinations *= hi - lo + 1
        total_accepted_combinations += combinations
    print(f"Total number of accepted rating combinations: {total_accepted_combinations}")

# Run the solutions for both parts
part1()
part2()


Total sum of ratings for accepted parts: 368523
Total number of accepted rating combinations: 124167549767307
